In [1]:
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock2 import tools
from block2 import SU2 as block2_SU2
from block2 import SZ as block2_SZ
from pathlib import Path
import CAS.dmrghandler.src.dmrghandler.dmrg_calc_prepare
import CAS.dmrghandler.src.dmrghandler.qchem_dmrg_calc
import CAS.dmrghandler.src.dmrghandler as dmrghandler
from CAS.dmrghandler.src.dmrghandler.dmrg_looping import dmrg_central_loop
from DMRG_simulation.dmrg_simulation import get_ground_state_fci
import numpy as np
import CAS_Cropping.saveload_utils as sl
import CAS_Cropping.ferm_utils as feru
import CAS_Cropping.var_utils as varu
import pyscf
import scipy
import openfermion as of

ModuleNotFoundError: No module named 'CAS'

In [ ]:
def hubbard_obt(n, t):
    tensor = np.zeros((n, n))
    
    for i in range(n-1):
        tensor[i+1, i] = -t
        tensor[i, i+1] = -t

    return [tensor, tensor]
    
def hubbard_tbt( n, U):
    tensor = np.zeros((n, n, n, n))
    for i in range(n):
        tensor[i, i, i, i] = U
        
    zero_tensor = np.zeros((n, n, n, n))
    return [zero_tensor, tensor, zero_tensor]

def hubbard_obt_2(n, t, s):
    tensor = np.zeros((n, n))
    for j in range(1, s+1):
        for i in range(n-j):
            tensor[i+j, i] = -t
            tensor[i, i+j] = -t

    return [tensor, tensor]

def hubbard_tbt_2(n, U):
    tensor = np.zeros((n, n, n, n))
    for i in range(n):
        tensor[i, i, i, i] = U
        
    zero_tensor = np.zeros((n, n, n, n))
    return [zero_tensor, tensor, zero_tensor]

def hubbard_obt_3(n, t, s):
    tensor = np.zeros((n, n))
    
    for i in range(n-s):
        tensor[i+s, i] = -t
        tensor[i, i+s] = -t

    return [tensor, tensor]

def hubbard_tbt_3(n, U, mu):
    tensor = np.zeros((n, n, n, n))
    mu_tensor = np.zeros((n, n, n, n))
    for i in range(n):
        tensor[i, i, i, i] = U
        for j in range(n):
            mu_tensor[i, i, j, j] = - mu
    
    
        
    zero_tensor = np.zeros((n, n, n, n))
    return [zero_tensor, tensor + mu_tensor, zero_tensor]

In [2]:
N = 6
t = 1
U = 1
s = 1
mu = 0
zero_tensor = np.zeros((N, N, N, N))
one_body_tensor = hubbard_obt_2(N, t, s)
two_body_tensor = hubbard_tbt_2(N, U)

NameError: name 'np' is not defined

In [7]:
print(one_body_tensor[0])

[[ 0. -1.  0.  0.]
 [-1.  0. -1.  0.]
 [ 0. -1.  0. -1.]
 [ 0.  0. -1.  0.]]


In [10]:
from DMRG_simulation.format_dmrg.format_dmrg_param import get_dmrg_param, get_dmrg_process_param


default_final_bond_dim = 1
default_sweep_schedule_bond_dims = [default_final_bond_dim] * 4 + [
    default_final_bond_dim+10
] * 4
default_sweep_schedule_noise = [1e-10] * 4 + [1e-10] * 4 + [1e-15]
default_sweep_schedule_davidson_threshold = [1e-20] * 8

init_state_bond_dimension = 10
max_num_sweeps = 200
energy_convergence_threshold = 1e-8
sweep_schedule_bond_dims = default_sweep_schedule_bond_dims
sweep_schedule_noise = default_sweep_schedule_noise
sweep_schedule_davidson_threshold = (
    default_sweep_schedule_davidson_threshold
)
nuc_rep_energy = 0
    
num_orbitals = N
num_electrons = N
num_unpaired = 0
multiplicity = 1

dmrg_process_param = get_dmrg_process_param(
    init_state_bond_dimension, max_num_sweeps, energy_convergence_threshold,
    sweep_schedule_bond_dims, sweep_schedule_noise,
    sweep_schedule_davidson_threshold)
dmrg_param = get_dmrg_param(
    num_orbitals, num_electrons, num_unpaired,
    multiplicity, dmrg_process_param)

result = dmrghandler.qchem_dmrg_calc.single_qchem_dmrg_calc(one_body_tensor, two_body_tensor, dmrg_param)
mps = result['ket_optimized']
print(result)

dtrie finished 0.00046050000000263935
Number of DET =         36 (cutoff =    1e-308)
Sum of weights of included DET =    1.000000000000000

DET          0 a2b0  =    0.370958331870666
DET          1 baab  =   -0.361077135183920
DET          2 ba02  =   -0.345960103392549
DET          3 a02b  =   -0.313850492422499
DET          4 0b2a  =    0.303870269196771
DET          5 b0a2  =    0.302405063594481
DET          6 0ab2  =    0.242599457927919
DET          7 a0b2  =    0.209903724955748
DET          8 b2a0  =   -0.175337236441715
DET          9 aabb  =   -0.160196864848492
DET         10 abab  =    0.140832753668588
DET         11 20ab  =    0.136827131396391
DET         12 2020  =   -0.131105014413130
DET         13 ba20  =   -0.125801592954510
DET         14 ab20  =   -0.117377467566193
DET         15 2a0b  =    0.111428575624033
DET         16 2200  =   -0.094121501035499
DET         17 02ba  =    0.084441136524223
DET         18 0220  =   -0.080744968540574
DET         19 2b0a  = 

In [9]:
loop_results = dmrg_central_loop(one_body_tensor, two_body_tensor, dmrg_param, 100, 1e6, 1e-15   , "./mps_directory", verbosity=2, move_mps_to_final_storage_path=None)

integral cutoff error =  0.0
mpo terms =         20

Build MPO | Nsites =     4 | Nterms =         20 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     4 .. Mmpo =     6 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.001 T = 0.014
 Site =     1 /     4 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.006
 Site =     2 /     4 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.003
 Site =     3 /     4 .. Mmpo =     1 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.002
Ttotal =      0.025 Tmvc-total = 0.001 MPO bond dimension =     6 MaxDW = 0.00e+00
NNZ =           34 SIZE =           84 SPT = 0.5952

Rank =     0 Ttotal =      0.051 MPO method = FastBipartite bond dimension =       6 NNZ =           34 SIZE =           84 SPT = 0.5952
dtrie finished 0.005228999999999928
Number of DET =         36 (cutoff =    1e-308)
Sum of weights of included DET =    1.000000000000000

DET          0 a2b0  =    0.370

/Users/rick/planted_solutions/planted_solutions/CAS/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =         20

Build MPO | Nsites =     4 | Nterms =         20 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     4 .. Mmpo =     6 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     4 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.001
 Site =     2 /     4 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.001
 Site =     3 /     4 .. Mmpo =     1 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.003 Tmvc-total = 0.000 MPO bond dimension =     6 MaxDW = 0.00e+00
NNZ =           34 SIZE =           84 SPT = 0.5952

Rank =     0 Ttotal =      0.008 MPO method = FastBipartite bond dimension =       6 NNZ =           34 SIZE =           84 SPT = 0.5952
dtrie finished 0.00026945800000532927
Number of DET =         36 (cutoff =    1e-308)
Sum of weights of included DET =    1.000000000000000

DET          0 aabb  =   -0.3

/Users/rick/planted_solutions/planted_solutions/CAS/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =         20

Build MPO | Nsites =     4 | Nterms =         20 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     4 .. Mmpo =     6 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     4 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.001
 Site =     2 /     4 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.000
 Site =     3 /     4 .. Mmpo =     1 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.002 Tmvc-total = 0.000 MPO bond dimension =     6 MaxDW = 0.00e+00
NNZ =           34 SIZE =           84 SPT = 0.5952

Rank =     0 Ttotal =      0.006 MPO method = FastBipartite bond dimension =       6 NNZ =           34 SIZE =           84 SPT = 0.5952
dtrie finished 0.0002601249999969468
Number of DET =         36 (cutoff =    1e-308)
Sum of weights of included DET =    1.000000000000000

DET          0 aabb  =   -0.35

/Users/rick/planted_solutions/planted_solutions/CAS/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =         20

Build MPO | Nsites =     4 | Nterms =         20 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     4 .. Mmpo =     6 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.000
 Site =     1 /     4 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.001
 Site =     2 /     4 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.001
 Site =     3 /     4 .. Mmpo =     1 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.002 Tmvc-total = 0.000 MPO bond dimension =     6 MaxDW = 0.00e+00
NNZ =           34 SIZE =           84 SPT = 0.5952

Rank =     0 Ttotal =      0.005 MPO method = FastBipartite bond dimension =       6 NNZ =           34 SIZE =           84 SPT = 0.5952
dtrie finished 0.0002547919999926762
Number of DET =         36 (cutoff =    1e-308)
Sum of weights of included DET =    1.000000000000000

DET          0 aabb  =   -0.35

/Users/rick/planted_solutions/planted_solutions/CAS/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =         20

Build MPO | Nsites =     4 | Nterms =         20 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     4 .. Mmpo =     6 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     4 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.001
 Site =     2 /     4 .. Mmpo =     6 DW = 0.00e+00 NNZ =       11 SPT = 0.6944 Tmvc = 0.000 T = 0.001
 Site =     3 /     4 .. Mmpo =     1 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.002
Ttotal =      0.005 Tmvc-total = 0.000 MPO bond dimension =     6 MaxDW = 0.00e+00
NNZ =           34 SIZE =           84 SPT = 0.5952

Rank =     0 Ttotal =      0.013 MPO method = FastBipartite bond dimension =       6 NNZ =           34 SIZE =           84 SPT = 0.5952
dtrie finished 0.00028012499998908424
Number of DET =         36 (cutoff =    1e-308)
Sum of weights of included DET =    1.000000000000000

DET          0 aabb  =   -0.3

/Users/rick/planted_solutions/planted_solutions/CAS/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


In [12]:
symmetry_type = SymmetryTypes.SZ
stack_mem = 10*1024*1024*1024
stack_mem_ratio = 0.5
num_threads = 3
restart_dir=Path("./temp")

In [13]:
mps_dir = Path("./mps_directory/mps_storage/dmrg_loop_005_ket_optimized")
driver = DMRGDriver(
    stack_mem=stack_mem,
    scratch=str(mps_dir),
    clean_scratch=True,  # Default value
    restart_dir=str(restart_dir),
    n_threads=num_threads,
    # n_mkl_threads=n_mkl_threads,  # Default value is 1
    symm_type=symmetry_type,
    mpi=None,  # Default value
    stack_mem_ratio=stack_mem_ratio,  # Default value 0.4
    fp_codec_cutoff=1e-120,  # Default value 1e-16,
)

In [14]:
tools.init(block2_SZ)
mps = tools.loadMPSfromDir(mps_info=None, mpsSaveDir=mps_dir)

cp: mps_directory/mps_storage/dmrg_loop_005_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 and mps_directory/mps_storage/dmrg_loop_005_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_005_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 and mps_directory/mps_storage/dmrg_loop_005_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_005_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 and mps_directory/mps_storage/dmrg_loop_005_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_005_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 and mps_directory/mps_storage/dmrg_loop_005_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_005_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.2 and mps_directory/mps_storage/dmrg_loop_005_ket_optimized/F.MPS.INFO.ket_

In [15]:
driver.initialize_system(n_sites=num_orbitals, n_elec=num_electrons, spin=num_unpaired, orb_sym=None)

In [16]:
csfs, coeffs = driver.get_csf_coefficients(mps, cutoff=1e-308, iprint=1)


mps center changed (temporarily)
dtrie finished 0.0022878750000074888
Number of DET =         34 (cutoff =    1e-308)
Sum of weights of included DET =    1.000000000000000

DET          0 baba  =    0.316039565245460
DET          1 abab  =    0.316039565245446
DET          2 baab  =   -0.248415556252741
DET          3 abba  =   -0.248415556252738
DET          4 20ba  =   -0.221603479857562
DET          5 20ab  =    0.221603479857558
DET          6 02ba  =   -0.221603479857551
DET          7 02ab  =    0.221603479857547
DET          8 ba20  =   -0.220358119666037
DET          9 ba02  =   -0.220358119666035
DET         10 ab20  =    0.220358119666030
DET         11 ab02  =    0.220358119666028
DET         12 2020  =    0.173023945653111
DET         13 2002  =    0.173023945653109
DET         14 0220  =    0.173023945653102
DET         15 0202  =    0.173023945653101
DET         16 b02a  =    0.120318621864796
DET         17 b20a  =    0.120318621864795
DET         18 a02b  =   -0.1203186

In [17]:
mpo = driver.get_qc_mpo(h1e=one_body_tensor, g2e=two_body_tensor, ecore=nuc_rep_energy, iprint=0)
ener = driver.expectation(mps, mpo, mps)
print(ener)

bra = driver.get_random_mps(tag="GS5", bond_dim=250, nroots=1)
applied_mps = driver.multiply(bra ,mpo, mps)
print(applied_mps)

variance = np.sqrt(abs(applied_mps ** 2 - ener ** 2))
print(f"Variance: {variance}")
print(f"Variance ratio: {variance / ener}")

-3.5584587088811563
3.569947116426202
Variance: 0.286171332015646
Variance ratio: -0.08042002322562383
